# Tensorboard and Optuna

* PyTorch [website](https://pytorch.org/)
* PyTorch [Tutorials](https://pytorch.org/tutorials/)
* TensorBoard [website](https://www.tensorflow.org/tensorboard)
* PyTorch [Tensorboard](https://pytorch.org/docs/stable/tensorboard.html) docs
* PyTorch TensorBoard [Tutorial](https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html)

# TensorBoard
Load the TensorBoard notebook extension

*Enable 3rd party cookies in your browser settings. Use Google Chrome.*

In [20]:
%load_ext tensorboard
logs_dir = './logs/'
%tensorboard --logdir {logs_dir} --host localhost

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 21892), started 20:03:45 ago. (Use '!kill 21892' to kill it.)

In [49]:
#!rm -r {logs_dir}

"rm" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [2]:
import time
import numpy as np
from torch.utils.tensorboard import SummaryWriter

# writer = SummaryWriter(log_dir=logs_dir+'experiment_02', flush_secs=2)

# for epoch in range(1, 100):
#     writer.add_scalar('Loss/train', np.random.random()/epoch, epoch)
#     writer.add_scalar('Loss/test', np.random.random()/epoch, epoch)
#     writer.add_scalar('Accuracy/train', np.random.random()*epoch, epoch)
#     writer.add_scalar('Accuracy/test', np.random.random()*epoch, epoch)
#     time.sleep(0.25)
# writer.close()

## Homework 04


All tasks could be done together.
1. (3) Use Optuna to tune hyperparameters. Decrease error rate of your best original model (Homework 03) by 20% at least. (E.g., your best model accuracy was 85%, the error rate was 100%-85%=15%; hence, you need to find hyperparameters to achieve 100%-15%*(100%-80%)=88% accuracy)
2. (1) Use TensorBoard to log each model training process (Train and Val loss and accuracy) (*.add_scalar*) and save hyperparameters of each model examined by Optuna (*.add_hparams*).
3. (1) Save visualization of each model architecture examined by Optuna to TensorBoard. (*.add_graph*)
4. \* (2) Save up to 3 misclassified images from the test dataset of each class of each model architecture examined by Optuna to TensorBoard.

In [3]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

import matplotlib.pyplot as plt

import math
import optuna

In [4]:
def get_loaders(batch_size=128, num_workers=2, transform=transforms.ToTensor()):
    train = datasets.CIFAR10('../data', train=True, download=True, transform=transform)
    test = datasets.CIFAR10('../data', train=False, download=True, transform=transform)
    torch.manual_seed(123)  # To ensure the same sampling during each experiment
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=num_workers, shuffle=False)
    return train_loader, test_loader

In [5]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        
        #ResNet
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        
        #Residual block 1
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(16)
        
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(16)
        self.conv5 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(16)
        
        
        #Residual block 2
        self.conv6 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.bn6 = nn.BatchNorm2d(32)
        self.conv7 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn7 = nn.BatchNorm2d(32)
        
        self.conv8 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.bn8 = nn.BatchNorm2d(32)
        
        self.conv9 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn9 = nn.BatchNorm2d(32)
        self.conv10 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn10 = nn.BatchNorm2d(32)
        
        
        #Residual block 3
        self.conv11 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1)
        self.bn11 = nn.BatchNorm2d(64)
        self.conv12 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn12 = nn.BatchNorm2d(64)
        
        self.conv13 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1)
        self.bn13 = nn.BatchNorm2d(64)
        
        self.conv14 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn14 = nn.BatchNorm2d(64)
        self.conv15 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn15 = nn.BatchNorm2d(64)
        
        
        #Final
        self.pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, 10)
    
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x, inplace=True)
        
        
        #Residual block 1
        
        residual = x
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x, inplace=True)
        x = self.conv3(x)
        x = self.bn3(x)
        x += residual
        x = F.relu(x, inplace=True)
        
        residual = x
        x = self.conv4(x)
        x = self.bn4(x)
        x = F.relu(x, inplace=True)
        x = self.conv5(x)
        x = self.bn5(x)
        x += residual
        x = F.relu(x, inplace=True)
        
        
        #Residual block 2
        
        residual = x
        x = self.conv6(x)
        x = self.bn6(x)
        x = F.relu(x, inplace=True)
        x = self.conv7(x)
        x = self.bn7(x)
        residual = self.conv8(residual)
        residual = self.bn8(residual)
        x += residual
        x = F.relu(x, inplace=True)
        
        residual = x
        x = self.conv9(x)
        x = self.bn9(x)
        x = F.relu(x, inplace=True)
        x = self.conv10(x)
        x = self.bn10(x)
        x += residual
        x = F.relu(x, inplace=True)
        
        
        #Residual block 3
        
        residual = x
        x = self.conv11(x)
        x = self.bn11(x)
        x = F.relu(x, inplace=True)
        x = self.conv12(x)
        x = self.bn12(x)
        residual = self.conv13(residual)
        residual = self.bn13(residual)
        x += residual
        x = F.relu(x, inplace=True)
        
        residual = x
        x = self.conv14(x)
        x = self.bn14(x)
        x = F.relu(x, inplace=True)
        x = self.conv15(x)
        x = self.bn15(x)
        x += residual
        x = F.relu(x, inplace=True)
        
        
        #Final
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.log_softmax(x, 1)
        
        return x

In [6]:
def train(model, device, train_loader, optimizer):
    model.train()
    train_loss = 0
    correct = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, target)
        loss.backward()
        optimizer.step()
        train_loss += F.nll_loss(out, target, reduction='sum').item()
        
        _, predicted = torch.max(out, 1)
        correct += (predicted == target).sum().item()
    
    return train_loss / len(train_loader.dataset), 100*correct/len(train_loader.dataset)


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            out = model(data)
            loss = F.nll_loss(out, target, reduction='sum')
            test_loss += loss.item()
            
            _, predicted = torch.max(out, 1)
            correct += (predicted == target).sum().item()
            
    return test_loss / len(test_loader.dataset), 100*correct/len(test_loader.dataset)

## Результаты обучения с исходными гиперпараметрами

In [9]:
epochs = 80
device = torch.device('cuda:0')

batch_size = 128
lr = 1e-3

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_loader, test_loader = get_loaders(batch_size=batch_size, transform=transform, num_workers=0)
model = ResNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max')

min_loss = 100
max_early_stop = 15
early_stop = 0


for epoch in range(epochs):
    train_loss, train_accuracy = train(model, device, train_loader, optimizer)
    test_loss, test_accuracy = test(model, device, test_loader)
    scheduler.step(test_accuracy)

    #early stop
    if test_loss > min_loss:
        early_stop += 1
    else:
        min_loss = test_loss
        early_stop = 0
    if early_stop == max_early_stop:
        print("Model doesn't improve, early stop")
        break


    print("Epoch: {}, train loss: {}, train accuracy: {}, test loss: {}, test accuracy: {}".format(epoch, train_loss, train_accuracy, test_loss, test_accuracy))

Files already downloaded and verified
Files already downloaded and verified
Epoch: 0, train loss: 1.3198433351135255, train accuracy: 52.152, test loss: 1.1042257577896117, test accuracy: 60.43
Epoch: 1, train loss: 0.9174589726257324, train accuracy: 67.116, test loss: 1.1893796867370605, test accuracy: 58.72
Epoch: 2, train loss: 0.7514306256866455, train accuracy: 73.244, test loss: 0.986364881515503, test accuracy: 65.92
Epoch: 3, train loss: 0.6434364685821533, train accuracy: 77.396, test loss: 0.832281383895874, test accuracy: 71.85
Epoch: 4, train loss: 0.5700424662780762, train accuracy: 79.976, test loss: 0.6637911425590515, test accuracy: 77.3
Epoch: 5, train loss: 0.5098514769744873, train accuracy: 82.178, test loss: 0.6829347948074341, test accuracy: 75.82
Epoch: 6, train loss: 0.45761971324920653, train accuracy: 83.934, test loss: 0.7763535768508911, test accuracy: 74.88
Epoch: 7, train loss: 0.40921455169677734, train accuracy: 85.714, test loss: 0.7343721952438355, te

(100%-78%)*0.2 = 4.4%
Требуется увеличить значение точности на 4.4%

## Поиск оптимальных гиперпараметров

In [11]:
train_split_len = 10000
test_split_len = 4000

def get_loaders_batch(batch_size=128, num_workers=2, transform=transforms.ToTensor()):
    train = datasets.CIFAR10('../data', train=True, download=True, transform=transform)
    test = datasets.CIFAR10('../data', train=False, download=True, transform=transform)
    torch.manual_seed(123)  # To ensure the same sampling during each experiment
    train = torch.utils.data.random_split(train, [train_split_len, len(train)-train_split_len])[0]
    test = torch.utils.data.random_split(test, [test_split_len, len(test)-test_split_len])[0]
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=num_workers, shuffle=False)
    return train_loader, test_loader

Результаты на части датасета

In [14]:
epochs = 80
device = torch.device('cuda:0')

batch_size = 1500
lr = 1e-3

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train_loader, test_loader = get_loaders_batch(batch_size=batch_size, transform=transform, num_workers=0)
model = ResNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'max')

min_loss = 128
max_early_stop = 15
early_stop = 0


for epoch in range(epochs):
    train_loss, train_accuracy = train(model, device, train_loader, optimizer)
    test_loss, test_accuracy = test(model, device, test_loader)
    scheduler.step(test_accuracy)

    #early stop
    if test_loss > min_loss:
        early_stop += 1
    else:
        min_loss = test_loss
        early_stop = 0
    if early_stop == max_early_stop:
        print("Model doesn't improve, early stop")
        break


    print("Epoch: {}, train loss: {}, train accuracy: {}, test loss: {}, test accuracy: {}".format(epoch, train_loss, train_accuracy, test_loss, test_accuracy))

Files already downloaded and verified
Files already downloaded and verified
Epoch: 0, train loss: 2.141661865234375, train accuracy: 21.25, test loss: 2.322151611328125, test accuracy: 8.875
Epoch: 1, train loss: 1.7917584716796875, train accuracy: 34.19, test loss: 2.7609059448242186, test accuracy: 11.325
Epoch: 2, train loss: 1.6402809814453125, train accuracy: 40.01, test loss: 2.9300391845703126, test accuracy: 11.3
Epoch: 3, train loss: 1.5237365478515625, train accuracy: 44.03, test loss: 2.747302062988281, test accuracy: 15.05
Epoch: 4, train loss: 1.4361932373046875, train accuracy: 47.87, test loss: 2.5305416259765625, test accuracy: 19.675
Epoch: 5, train loss: 1.3608674682617188, train accuracy: 50.57, test loss: 1.7043411254882812, test accuracy: 40.025
Epoch: 6, train loss: 1.3018899291992188, train accuracy: 52.91, test loss: 2.24929931640625, test accuracy: 28.025
Epoch: 7, train loss: 1.23191181640625, train accuracy: 55.72, test loss: 1.6827589111328125, test accuracy

In [15]:
#отдельная функция трейна с эпохами

def epoch_train(model, params, trial_n):
    
    experiment = 'trial_'+str(trial_n)
    writer = SummaryWriter(log_dir=logs_dir+experiment, flush_secs=2)
    
    epochs = 80
    max_early_stop = 15
    min_loss = 100
    early_stop = 0
    batch_size = 1500
    
    lr = params["lr"]
    weight_decay = params["weight_decay"]
    amsgrad = params["amsgrad"]
    patience = params["patience"]
    
    train_loader, test_loader = get_loaders_batch(batch_size=batch_size, transform=transform, num_workers=0)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay = weight_decay, amsgrad = amsgrad)
    scheduler = ReduceLROnPlateau(optimizer, 'max', patience = patience)
    
    accuracy = 0
    for epoch in range(epochs):
        train_loss, train_accuracy = train(model, device, train_loader, optimizer)
        test_loss, test_accuracy = test(model, device, test_loader)
        scheduler.step(test_accuracy)
        
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/test', test_loss, epoch)
        writer.add_scalar('Accuracy/train', train_accuracy, epoch)
        writer.add_scalar('Accuracy/test', test_accuracy, epoch)

        #early stop
        if test_loss > min_loss:
            early_stop += 1
        else:
            min_loss = test_loss
            early_stop = 0
        if early_stop == max_early_stop:
            print("Model doesn't improve, early stop")
            accuracy = test_accuracy
            break
        if epoch==epochs-1:
            accuracy = test_accuracy
    
    writer.add_hparams(hparam_dict = params, metric_dict = {'accuracy':accuracy})
    writer.close()
    
    return accuracy

In [16]:
device = torch.device('cuda:0')

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def objective(trial):
    
    model = ResNet().to(device)
    
    amsgrad = trial.suggest_categorical("amsgrad",[True, False])
    lr = trial.suggest_categorical("lr", [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
    weight_decay = trial.suggest_categorical("weight_decay", [0, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
    patience = trial.suggest_int("patience", 4, 10)
    
    params = {"lr":lr, "weight_decay":weight_decay, "amsgrad": amsgrad, "patience": patience}
    
    accuracy = epoch_train(model, params, trial.number)
    
    print(accuracy)
    return accuracy

In [17]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials = 60)

[I 2021-03-02 17:13:12,629] A new study created in memory with name: no-name-6ef50447-e555-426c-b14f-b0adada735d2


Files already downloaded and verified
Files already downloaded and verified


[I 2021-03-02 17:20:41,242] Trial 0 finished with value: 67.775 and parameters: {'amsgrad': True, 'lr': 0.01, 'weight_decay': 1e-07, 'patience': 9}. Best is trial 0 with value: 67.775.


Model doesn't improve, early stop
67.775
Files already downloaded and verified
Files already downloaded and verified


[I 2021-03-02 17:32:27,872] Trial 1 finished with value: 36.975 and parameters: {'amsgrad': True, 'lr': 1e-05, 'weight_decay': 0.0001, 'patience': 6}. Best is trial 0 with value: 67.775.


...

Model doesn't improve, early stop
69.525
Files already downloaded and verified
Files already downloaded and verified


[I 2021-03-03 01:33:21,125] Trial 58 finished with value: 66.05 and parameters: {'amsgrad': False, 'lr': 0.01, 'weight_decay': 0.01, 'patience': 5}. Best is trial 10 with value: 71.725.


Model doesn't improve, early stop
66.05
Files already downloaded and verified
Files already downloaded and verified


[I 2021-03-03 01:42:46,244] Trial 59 finished with value: 65.35 and parameters: {'amsgrad': False, 'lr': 0.01, 'weight_decay': 0.001, 'patience': 4}. Best is trial 10 with value: 71.725.


Model doesn't improve, early stop
65.35


Гиперпараметры, выдающие лучший результат согласно тестам optuna 

In [18]:
def get_loaders(batch_size=128, num_workers=2, transform=transforms.ToTensor()):
    train = datasets.CIFAR10('../data', train=True, download=True, transform=transform)
    test = datasets.CIFAR10('../data', train=False, download=True, transform=transform)
    torch.manual_seed(123)  # To ensure the same sampling during each experiment
    
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=num_workers, shuffle=False)
    return train_loader, test_loader

In [19]:
epochs = 80
device = torch.device('cuda:0')

batch_size = 1500

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_loader, test_loader = get_loaders(batch_size=batch_size, transform=transform, num_workers=0)
model = ResNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay = 0.001, amsgrad = False)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience = 4)

min_loss = 100
max_early_stop = 15
early_stop = 0


for epoch in range(epochs):
    train_loss, train_accuracy = train(model, device, train_loader, optimizer)
    test_loss, test_accuracy = test(model, device, test_loader)
    scheduler.step(test_accuracy)

    #early stop
    if test_loss > min_loss:
        early_stop += 1
    else:
        min_loss = test_loss
        early_stop = 0
    if early_stop == max_early_stop:
        print("Model doesn't improve, early stop")
        break


    print("Epoch: {}, train loss: {}, train accuracy: {}, test loss: {}, test accuracy: {}".format(epoch, train_loss, train_accuracy, test_loss, test_accuracy))            

Files already downloaded and verified
Files already downloaded and verified
Epoch: 0, train loss: 1.9842961401367187, train accuracy: 24.394, test loss: 1.9445603637695312, test accuracy: 27.61
Epoch: 1, train loss: 1.6038608068847657, train accuracy: 39.682, test loss: 2.9688794189453125, test accuracy: 18.33
Epoch: 2, train loss: 1.3838228686523437, train accuracy: 49.03, test loss: 2.70900595703125, test accuracy: 26.76
Epoch: 3, train loss: 1.1870370275878905, train accuracy: 57.112, test loss: 1.6661898803710937, test accuracy: 39.91
Epoch: 4, train loss: 1.04872478515625, train accuracy: 62.384, test loss: 1.4851636840820313, test accuracy: 49.4
Epoch: 5, train loss: 0.9413260284423828, train accuracy: 66.484, test loss: 1.4462117797851564, test accuracy: 49.71
Epoch: 6, train loss: 0.8621785192871094, train accuracy: 69.376, test loss: 2.21353056640625, test accuracy: 32.05
Epoch: 7, train loss: 0.7834112194824219, train accuracy: 72.386, test loss: 1.4390924926757813, test accu

Гиперпараметры, подобранные на части датасета, дали увеличение точности и на всем датасете.